In [2]:
import torch
from torch import nn
from torchrl.data import BoundedTensorSpec
from torchrl.modules.distributions.continuous import NormalParamWrapper, TanhNormal
from torchrl.modules.tensordict_module.actors import ProbabilisticActor, ValueOperator
from torchrl.modules.tensordict_module.common import SafeModule
from torchrl.objectives.sac import SACLoss
from tensordict import TensorDict
from torchrl.objectives.sac import SACLoss
from tensordict import TensorDict

In [ ]:
n_act, n_obs = 15,512
spec = BoundedTensorSpec(-torch.ones(n_act), torch.ones(n_act), (n_act,))

net = NormalParamWrapper(nn.Linear(n_obs, 2 * n_act))
module = SafeModule(net, in_keys=["observation"], out_keys=["loc", "scale"])
actor = ProbabilisticActor(
    module=module,
    in_keys=["loc", "scale"],
    spec=spec,
    distribution_class=TanhNormal)

In [ ]:
class ValueClass(nn.Module):
    def __init__(self,):
        super().__init__()
        self.linear = nn.Linear(n_obs + n_act, 1)
    def forward(self, obs, act):
        return self.linear(torch.cat([obs, act], -1))
    
module = ValueClass()

qvalue = ValueOperator(
    module=module,
    in_keys=['observation', 'action'])

loss = SACLoss(actor, qvalue)

In [ ]:
# batch = [2, ]
# action = spec.rand(batch)
# buffer_lazytensor = ReplayBuffer(storage=LazyTensorStorage(size))
# from torchrl.envs.transforms import CatTensors

In [3]:
import torch
from tensordict import TensorDict
from torchrl.data import ReplayBuffer
from torchrl.data.replay_buffers.samplers import PrioritizedSampler
from torchrl.data import LazyMemmapStorage, LazyTensorStorage, ListStorage
from torchrl.data import TensorDictReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
size = 100

In [ ]:
from torchrl.envs.transforms import (
    Compose,
    GrayScale,
    Resize,
    ToTensorImage,
    TransformedEnv,
)
a =  Compose(
        ToTensorImage(in_keys=["pixels"], out_keys=["pixels_trsf"]),
        Resize(in_keys=["pixels_trsf"], w=64, h=64),
        GrayScale(in_keys=["pixels_trsf"]),
    )

In [4]:
from envs.photo_env import PhotoEnhancementEnv

Encoding testing data ...


  0%|          | 0/4 [00:00<?, ?it/s]

finished...
Encoding training data ...


  0%|          | 0/36 [00:00<?, ?it/s]

finished...


In [5]:
import torch
env = PhotoEnhancementEnv(64)
state = env.reset()
action = torch.rand(64,15)
next_state,rewards, dones  = env.step(action)

In [ ]:
from torchrl.envs.transforms import (
    Compose,
    GrayScale,
    Resize,
    ToTensorImage,
    TransformedEnv,CatTensors
)
transform = Compose(
    CatTensors(in_keys = [("state","encoded_enhanced_image"),("state","encoded_source")], out_key = ('state','encoded_observation')),
    CatTensors(in_keys = [("next_state","encoded_enhanced_image"),("next_state","encoded_source")], out_key = ('next_state','encoded_observation'))
)
transform = Compose(
    CatTensors(in_keys = ["encoded_enhanced_image","encoded_source"], out_key = 'encoded_observation'),
)

In [6]:
buffer_ = TensorDictReplayBuffer(
    storage=LazyMemmapStorage(size,), sampler=SamplerWithoutReplacement()
)

In [7]:
transition ={
    'state':state,
    'action':action,
    'next_state':next_state,
    'rewards':rewards,
    'done':dones,
}

In [14]:
td = TensorDict(transition ,batch_size=[state.shape[0]])

In [26]:
buffer_.extend(td)

tensor([64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
        82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99,
         0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [40]:
rewards.mean()

tensor(-0.4233)

In [38]:
buffer_.sample(12)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([12, 15]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([12]), device=cpu, dtype=torch.bool, is_shared=False),
        index: Tensor(shape=torch.Size([12]), device=cpu, dtype=torch.int64, is_shared=False),
        next_state: Tensor(shape=torch.Size([12, 1024]), device=cpu, dtype=torch.float32, is_shared=False),
        rewards: Tensor(shape=torch.Size([12]), device=cpu, dtype=torch.float32, is_shared=False),
        state: Tensor(shape=torch.Size([12, 1024]), device=cpu, dtype=torch.float32, is_shared=False)},
    batch_size=torch.Size([12]),
    device=cpu,
    is_shared=False)